# Introduction to Relational Databases in Python
- learn the basics of using SQL with Python
- Python SQL toolkit SQLAlchemy provides an accessible and intuitive way to query, build & write to SQLite, MySQL and Postgresql databases (among many others)

Outline:
1. Basics of Relational Databases
2. Applying Filtering, Ordering, and Grouping to Queries
3. Advanced SQLAlchemy Queries
4. Creating and Manipulating your own Databases
5. Project: Putting it all together

# 1. Basics of Relational Databases

- relational databases are made up of tables
- Table consist of columns and rows
- Tables can be related (thus "Relational" databases)


SQLAlchemy
- 2 main pieces
    1. Core (Relational Model focused)
    2. ORM (User Data Model focused)

Many types of databases - SQLAlchemy can connect to
- SQLite
- PostgreSQL
- MySQL
- MS SQL
- Oracle
- Many more

Connecting to a database
- Engine - common interface to the db from SQLAlchemy
- Connection string: all the details required to find the database (and login if necessary)

Note on connection strings
- example: 'sqlite:///census_nyc.sqlite'
    - Driver+Dialect: sqlite:///
    - Filename: census_nyc.sqlite
    

## 1.1 Connecting to a DB - sqlalchemy

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///census_nyc.sqlite')
connection = engine.connect()


## 1.2 What's in the database?
- before querying your db, check what's in it
    - what the tables are

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///census_nyc.sqlite')

# check table
print(engine.table_names())
# ['census', 'state_fact']

## 1.3 Reflection
- access the table
- Reflection reads db and builds SQLAlchemy Table objects

In [ ]:
from sqlalchemy import MetaData, Table
metadata = MetaData()
census = Table('census', metadata, autoload=True, 
               autoload_with=engine)

# view column names and data types
print(repr(census))

## 1.4 Examples: Engines and Connection Strings
Alright, it's time to create your first engine! An engine is just a common interface to a database, and the information it requires to connect to one is contained in a connection string, such as sqlite:///census_nyc.sqlite. Here, sqlite is the database driver, while census_nyc.sqlite is a SQLite file contained in the local directory.

You can learn a lot more about connection strings in the SQLAlchemy documentation. http://docs.sqlalchemy.org/en/latest/core/engines.html#database-urls

Your job in this exercise is to create an engine that connects to a local SQLite file named census.sqlite. Then, print the names of the tables it contains using the .table_names() method. Note that when you just want to print the table names, you do not need to use engine.

### 

### 

### 

### 

### 

### 

# 2. Applying Filtering, Ordering, and Grouping to Queries

### 

### 

### 

### 

### 

### 

### 

### 

### 

### 

# 3. Advanced SQLAlchemy Queries

### 

### 

### 

### 

### 

### 

### 

### 

### 

### 

# 4. Creating and Manipulating your own Databases

### 

### 

### 

### 

### 

### 

### 

### 

### 

### 

# 5. Project: Putting it all together

### 

### 

### 

### 

### 

### 

### 

### 

### 

### 